In [174]:
import numpy as np
import pandas as pd
import requests
import re
import time
import pickle
import datetime

from bs4 import BeautifulSoup as bs


In [169]:
#네이버 도전웹툰 1~1000page의 모든 웹툰 제목(title), 링크(link), 평점(star)를 수집
def get_challenge_webtoon(page):
    url = 'https://comic.naver.com/genre/challenge.nhn?m=list&page={}'.format(page)
    response = requests.get(url)
    webtoons = bs(response.content, 'html.parser')
    
    domain = 'https://comic.naver.com'
    for webtoon in webtoons.select('.challengeInfo'):
        table.append({
            
            'title' : webtoon.select('a')[0].text.strip(),
            'link' : domain + webtoon.select('a')[0].get('href'),
            'star' : webtoon.select_one('.star').get('title')
        
        })

#link를 받아 에피소드의 개수를 체크함 (첫번째 url = 마지막 에피소드 url에 회차정보가 담겨 있음)       
def check_num_episode(title_link):
    response = requests.get(title_link)
    last_episode_html = bs(response.content, 'html.parser').select_one('.title > a').get('href')
    last_episode_date = bs(response.content, 'html.parser').select_one('.num').text
    
    return last_episode_html[-4:], last_episode_date

In [166]:
table = []
for page in range(1,5):
    get_challenge_webtoon(page)

result = pd.DataFrame(table)
result.star = result.star.apply(lambda x: float(x.replace('점','')))
result['title_id'] = result.link.apply(lambda x:x[-6:])
result.tail()

,link,star,title,title_id
91,https://comic.naver.com/challenge/list.nhn?tit...,7.43,와이번티스 단편,738790
92,https://comic.naver.com/challenge/list.nhn?tit...,9.77,카페 오아시스,736681
93,https://comic.naver.com/challenge/list.nhn?tit...,8.00,토 란,689319
94,https://comic.naver.com/challenge/list.nhn?tit...,0.00,waii가 사는 세상,742289
95,https://comic.naver.com/challenge/list.nhn?tit...,9.98,우. 숨(숨은그림). 명.,742284


In [171]:
links[0]
check_num_episode(links[0])

('o=24', '2020.03.05')

In [172]:
num = []
date = []

links = result.link

for i in range(len(links)):
    try:
        num.append(check_num_episode(links[i])[0])
        date.append(check_num_episode(links[i])[1])
        
    except AttributeError:
        num.append('0')
        date.append('0')
    
    print(str(i+1) +'/{}'.format(len(links)))

result['num_episode'] = num
result['num_episode'] = result.num_episode.apply(lambda x: re.sub('[a-z=]','',x))
result['last_date'] = date

1/96
2/96
3/96
4/96
5/96
6/96
7/96
8/96
9/96
10/96
11/96
12/96
13/96
14/96
15/96
16/96
17/96
18/96
19/96
20/96
21/96
22/96
23/96
24/96
25/96
26/96
27/96
28/96
29/96
30/96
31/96
32/96
33/96
34/96
35/96
36/96
37/96
38/96
39/96
40/96
41/96
42/96
43/96
44/96
45/96
46/96
47/96
48/96
49/96
50/96
51/96
52/96
53/96
54/96
55/96
56/96
57/96
58/96
59/96
60/96
61/96
62/96
63/96
64/96
65/96
66/96
67/96
68/96
69/96
70/96
71/96
72/96
73/96
74/96
75/96
76/96
77/96
78/96
79/96
80/96
81/96
82/96
83/96
84/96
85/96
86/96
87/96
88/96
89/96
90/96
91/96
92/96
93/96
94/96
95/96
96/96


In [206]:
now = datetime.datetime.now()
now = datetime.datetime.date(now)

result['last_date'] = result['last_date'].apply(lambda x: datetime.datetime.date(datetime.datetime.strptime(x, '%Y.%m.%d')))
result['last_date'] = result['last_date'].apply(lambda x: (now - s).td)
result.head(2)

TypeError: strptime() argument 1 must be str, not datetime.date

In [209]:
result['last_date'] = result['last_date'].apply(lambda x: (now - x).days)
result.head(2)

,link,star,title,title_id,num_episode,last_date
0,https://comic.naver.com/challenge/list.nhn?tit...,8.88,Summon me now!,733109,3,0
1,https://comic.naver.com/challenge/list.nhn?tit...,9.10,댓글을 부르는 만화,740973,10,0


In [211]:
result

,link,star,title,title_id,num_episode,last_date
0,https://comic.naver.com/challenge/list.nhn?tit...,8.88,Summon me now!,733109,3,0
1,https://comic.naver.com/challenge/list.nhn?tit...,9.10,댓글을 부르는 만화,740973,10,0
2,https://comic.naver.com/challenge/list.nhn?tit...,0.00,가족,742487,1,0
3,https://comic.naver.com/challenge/list.nhn?tit...,9.35,심리하는 무당이,739200,11,0
4,https://comic.naver.com/challenge/list.nhn?tit...,0.00,aewrw,742486,1,0
5,https://comic.naver.com/challenge/list.nhn?tit...,0.00,쿠킹 OK,741533,2,0
6,https://comic.naver.com/challenge/list.nhn?tit...,8.89,나의 마인드,731298,31,0
7,https://comic.naver.com/challenge/list.nhn?tit...,6.87,햄스툰(어느순간 내 찍찍이가 사람으로..),711395,76,0
8,https://comic.naver.com/challenge/list.nhn?tit...,10.00,신발전쟁,742476,3,0
9,https://comic.naver.com/challenge/list.nhn?tit...,9.37,하얀 동화,704557,69,0


In [183]:
print(datetime.datetime.strptime(result.last_date[0], '%Y.%m.%d'))
# result.last_date[0]

2020-03-05 00:00:00


In [201]:
now = datetime.datetime.now()
now = datetime.datetime.date(now)
now

datetime.date(2020, 3, 5)

In [191]:
last = datetime.datetime.date(datetime.datetime.strptime(result.last_date[0], '%Y.%m.%d'))
last

datetime.date(2020, 3, 5)

In [203]:
td = now - last
td.days

0

In [ ]:
### 웹툰

In [89]:
url = 'https://comic.naver.com/genre/challenge.nhn?m=list&page=1'
domain = 'https://comic.naver.com'
response = requests.get(url)
webtoons = bs(response.content, 'html.parser')

In [113]:
malsook = webtoons.select('.challengeInfo')[7]

In [114]:
title_link = domain + malsook.select('a')[0].get('href')

In [147]:
title_link

'https://comic.naver.com/challenge/list.nhn?titleId=740398'

In [150]:
response = requests.get(title_link)
html = bs(response.content, 'html.parser')

In [151]:
html


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>꺄릌툰 :: 네이버 만화</title>
<meta content="꺄릌툰" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/user_contents_data/challenge_comic/2020/01/28/dnjfxns/title_thumbnail_220017_202x164.jpg" property="og:image"/>
<meta content="꺄릌!" property="og:description"/>
<meta content="https://comic.naver.com/challenge/list.nhn?titleId=740398" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<link href="/css/comic/common_20200220150639.css" rel="stylesheet" sty

In [152]:
last_episode = domain + html.select('td.title')[0].select_one('a').get('href')

In [153]:
last_episode

'https://comic.naver.com/challenge/detail.nhn?titleId=740398&no=6'

In [154]:
response = requests.get(last_episode)
html = bs(response.content, 'html.parser')
html


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>꺄릌툰 :: 네이버 만화</title>
<meta content="꺄릌툰 - 6화" property="og:title"/>
<meta content="https://shared-comic.pstatic.net/thumb/nas/user_contents_data/challenge_comic/2020/03/04/dnjfxns/thumbnail_600x3155f41d33a_4648_4bba_a798_943d909b77af_00004199.JPEG" property="og:image"/>
<meta content="6화" property="og:description"/>
<meta content="https://comic.naver.com/challenge/detail.nhn?titleId=740398&amp;no=6" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<meta content="tex

In [164]:
#조회수
html.select('.rt > dd.date')[1].text

('2020.03.04', '40')

In [146]:
html.select_one('.wt_viewer').select('img')[-1].get('id')[-2:]

'_6'

<img alt="흔한 한일부부의 그림일기" height="79" onerror="this.src='https://static-comic.pstatic.net/staticImages/COMICWEB/NAVER/img/common/non98_79.gif'" src="https://shared-comic.pstatic.net/thumb/nas/user_contents_data/challenge_comic/2020/02/16/jon_gattyan/title_thumbnail_171743_98x79.jpg" title="흔한 한일부부의 그림일기" width="98"/>

In [168]:
num,date = get_challenge_webtoon(1)

TypeError: 'NoneType' object is not iterable